In [4]:
# Homework 4
# Center of Mass Position and Velocity
# Hina Suzuki

In [6]:
# import modules
import numpy as np
import astropy.units as u
import astropy.table as tbl

from ReadFile import Read

In [7]:
class CenterOfMass:
# Class to define COM position and velocity properties 
# of a given galaxy and simulation snapshot

    def __init__(self, filename, ptype):
        ''' Class to calculate the 6-D phase-space position of a galaxy's center of mass using
        a specified particle type. 
            
            PARAMETERS
            ----------
            filename : `str`
                snapshot file
            ptype : `int; 1, 2, or 3`
                particle type to use for COM calculations
        '''
     
        # read data in the given file using Read
        self.time, self.total, self.data = Read(filename)                                                                                             

        #create an array to store indexes of particles of desired Ptype                                
        self.index = np.where(self.data['type'] == ptype)

        # store the mass, positions, velocities of only the particles of the given type
        # the following only gives the example of storing the mass
        self.m = self.data['m'][self.index]
        self.x = self.data['x'][self.index] 
        self.y = self.data['y'][self.index] 
        self.z = self.data['z'][self.index] 
        self.vx = self.data['vx'][self.index] 
        self.vy = self.data['vy'][self.index]
        self.vz = self.data['vz'][self.index] 

    def COMdefine(self,a,b,c,m):
        ''' Method to compute the COM of a generic vector quantity by direct weighted averaging.
        
        PARAMETERS
        ----------
        a : `float or np.ndarray of floats`
            first vector component
        b : `float or np.ndarray of floats`
            second vector component
        c : `float or np.ndarray of floats`
            third vector component
        m : `float or np.ndarray of floats`
            particle masses
        
        RETURNS
        -------
        a_com : `float`
            first component on the COM vector
        b_com : `float`
            second component on the COM vector
        c_com : `float`
            third component on the COM vector
        '''
       
        # xcomponent Center of mass
        a_com = np.sum(a*m) / np.sum(m)
        # ycomponent Center of mass
        b_com = np.sum(b*m) / np.sum(m)
        # zcomponent Center of mass
        c_com = np.sum(c*m) / np.sum(m)
        
        return a_com, b_com, c_com
    
    def COM_P(self, delta):
        '''Method to compute the position of the center of mass of the galaxy 
        using the shrinking-sphere method.

        PARAMETERS
        ----------
        delta : `float, optional`
            error tolerance in kpc. Default is 0.1 kpc
        
        RETURNS
        ----------
        p_COM : `np.ndarray of astropy.Quantity'
            3-D position of the center of mass in kpc
        '''                                                                     

        # Center of Mass Position                                                                                      
        ###########################                                                                                    

        # Try a first guess at the COM position by calling COMdefine                                                   
        x_COM, y_COM, z_COM = self.COMdefine(self.x, self.y, self.z, self.m)
        # compute the magnitude of the COM position vector.
        r_COM = np.sqrt(x_COM**2 + y_COM**2 + z_COM**2)


        # iterative process to determine the center of mass                                                            

        # change reference frame to COM frame                                                                          
        # compute the difference between particle coordinates                                                          
        # and the first guess at COM position
        x_new = self.x - x_COM
        y_new = self.y - y_COM
        z_new = self.z - z_COM
        r_new = np.sqrt(x_new**2 + y_new**2 + z_new**2)


        # find the max 3D distance of all particles from the guessed COM                                               
        # will re-start at half that radius (reduced radius)                                                           
        r_max = max(r_new)/2.0
        
        # pick an initial value for the change in COM position                                                      
        # between the first guess above and the new one computed from half that volume
        # it should be larger than the input tolerance (delta) initially
        change = 1000.0

        # start iterative process to determine center of mass position                                                 
        # delta is the tolerance for the difference in the old COM and the new one.    
        
        while (change > delta):
            # select all particles within the reduced radius (starting from original x,y,z, m)
            index2 = np.where(r_max > r_new)
            x2 = self.x[index2]
            y2 = self.y[index2]
            z2 = self.z[index2]
            m2 = self.m[index2]

            # Refined COM position:                                                                                    
            # compute the center of mass position using                                                                
            # the particles in the reduced radius
            x_COM2, y_COM2, z_COM2 = self.COMdefine(x2, y2, z2, m2)
            # compute the new 3D COM position
            r_COM2 = np.sqrt(x_COM2**2 + y_COM2**2 + z_COM2**2 )

            # determine the difference between the previous center of mass position                                    
            # and the new one.                                                                                         
            change = np.abs(r_COM - r_COM2)
           
            # Before loop continues, reset : r_max, particle separations and COM                                        

            # reduce the volume by a factor of 2 again                                                                 
            r_max /= 2.0
            # check this.                                                                                              
            #print ("maxR", r_max)                                                                                      

            # Change the frame of reference to the newly computed COM.                                                 
            # subtract the new COM
            x_new = self.x - x_COM2
            y_new = self.y - y_COM2
            z_new = self.z - z_COM2
            r_new = np.sqrt(x_new**2 + y_new**2 + z_new**2)

            # set the center of mass positions to the refined values                                                   
            x_COM = x_COM2
            y_COM = y_COM2
            z_COM = z_COM2
            r_COM = r_COM2

        # create an array (np.array) to store the COM position                                                                                                                                                       
        p_COM = np.array([x_COM, y_COM, z_COM])

       
        # set the correct units using astropy and round all values
        # and then return the COM positon vector
        p_COM = np.round([x_COM, y_COM, z_COM], 2) 
        return p_COM*u.kpc

        
        
    def COM_V(self, x_COM, y_COM, z_COM):
        ''' Method to compute the center of mass velocity based on the center of mass
        position.

        PARAMETERS
        ----------
        x_COM : 'astropy quantity'
            The x component of the center of mass in kpc
        y_COM : 'astropy quantity'
            The y component of the center of mass in kpc
        z_COM : 'astropy quantity'
            The z component of the center of mass in kpc
            
        RETURNS
        -------
        v_COM : `np.ndarray of astropy.Quantity'
            3-D velocity of the center of mass in km/s
        '''
        
        # the max distance from the center that we will use to determine 
        #the center of mass velocity                   
        rv_max = 15.0*u.kpc

        # determine the position of all particles relative to the center of mass position (x_COM, y_COM, z_COM)
        xV = self.x *u.kpc - x_COM
        yV = self.y *u.kpc - y_COM
        zV = self.z *u.kpc - z_COM
        rV = np.sqrt(xV**2 + yV**2 + zV**2)
        
        # determine the index for those particles within the max radius
        indexV = np.where(rV < rv_max)
        
        # determine the velocity and mass of those particles within the mas radius
        vx_new = self.vx[indexV]
        vy_new = self.vy[indexV]
        vz_new = self.vz[indexV]
        m_new =  self.m[indexV]
        
        # compute the center of mass velocity using those particles
        vx_COM, vy_COM, vz_COM = self.COMdefine(vx_new, vy_new, vz_new, m_new)
        
        # create an array to store the COM velocity round number into 2 decimal places
        v_COM = np.round([vx_COM, vy_COM, vz_COM], 2)

        # return the COM vector
        # set the correct units usint astropy
        return v_COM * u.km/u.s
     
    

In [10]:
# Create a Center of mass object for the MW, M31 and M33
MW_COM = CenterOfMass("MW_000.txt", 2)
M31_COM = CenterOfMass("M31_000.txt", 2)
M33_COM = CenterOfMass("M33_000.txt", 2)

## Questions

#1 COM positions of MW, M31, M33

In [18]:
# MW:   store the position and velocity COM

print("COM position and velocity vector of MW")
MW_COM_p = MW_COM.COM_P(0.1)
print(MW_COM_p)
MW_COM_v = MW_COM.COM_V(MW_COM_p[0], MW_COM_p[1], MW_COM_p[2])
print(MW_COM_v)

COM position and velocity vector of MW
[-2.05  2.96 -2.05] kpc
[ 1.42  7.59 -1.34] km / s


In [19]:
print("COM position and velocity vector of M31")
M31_COM_p = M31_COM.COM_P(0.1)
print(M31_COM_p)
M31_COM_v = M31_COM.COM_V(M31_COM_p[0], M31_COM_p[1], M31_COM_p[2])
print(M31_COM_v)

COM position and velocity vector of M31
[-377.67  611.44 -377.67] kpc
[ 73.37 -72.54  49.31] km / s


In [24]:
print("COM position and velocity vector of M33")
M33_COM_p = M33_COM.COM_P(0.1)
print(M33_COM_p)
M33_COM_v = M33_COM.COM_V(M33_COM_p[0], M33_COM_p[1], M33_COM_p[2])
print(M33_COM_v)

COM position and velocity vector of M33
[-476.26  491.46 -476.26] kpc
[ 44.43 101.84 142.37] km / s


#2 Separation and relative velocity of M31 and MW

In [28]:
# magnitude of current separation and velocity of MW and M31
# sepatation vector (x_M31 - x_MW, Y_M31-Y_MW, Z_M31-Z_MW)
# magnitude of separation = sqrt(x^2 + y^2 + z^2 )


separaion_M31_MW = M31_COM_p - MW_COM_p 
mag_separation_M31_MW = np.sqrt(separaion_M31_MW[0]**2 +separaion_M31_MW[1]**2 + separaion_M31_MW[2]**2 )
print(np.round(mag_separation_M31_MW, 3))

rel_vel_M31_MW = M31_COM_v - MW_COM_v 
mag_rel_vel_M31_MW = np.sqrt(rel_vel_M31_MW[0]**2 +rel_vel_M31_MW[1]**2 + rel_vel_M31_MW[2]**2 )
print(np.round(mag_rel_vel_M31_MW,3))

807.731 kpc
119.009 km / s


In lecture we learned the separation of MW and M31 is - 770kpc, relative velocity is 110km/s. My result is fairly close to this number. 

#3 Separation and relative velocity of M31 and M33

In [31]:
# magnitude of current separation and velocity of M33 and M31 

separaion_M31_M33 = M31_COM_p - M33_COM_p 
mag_separation_M31_M33 = np.sqrt(separaion_M31_M33[0]**2 +separaion_M31_M33[1]**2 + separaion_M31_M33[2]**2 )
print(np.round(mag_separation_M31_M33,3))

rel_vel_M31_M33 = M31_COM_v - M33_COM_v 
mag_rel_vel_M31_M33 = np.sqrt(rel_vel_M31_M33[0]**2 +rel_vel_M31_M33[1]**2 + rel_vel_M31_MW[2]**2 )
print(np.round(mag_rel_vel_M31_M33,3))

183.943 kpc
183.879 km / s


#4 Given that M31 and the MW are about to merge, why is the iterative process to determine the COM is important?

This iterative process helpes to find the change of COM due to collision/merge of two galaxy. 